In [11]:
import pickle, multiprocessing
import gensim 
from pprint import pprint

In [22]:
# Load docs
data = pickle.load(open("onetsoccode.p", "rb"))
documents = [x['text'].split() for x in data]

In [31]:
jobs = pickle.load(open('lookuptable', 'rb'))
jobs = {v['onetsoc_code']:v for k, v in jobs.items()}

# Preprocess

In [32]:
train_corpus = []

for idx, x in enumerate(data): 
    preprocess = gensim.utils.simple_preprocess(x['text'])
    tag = gensim.models.doc2vec.TaggedDocument(preprocess, [x['soc']])
    train_corpus.append(tag)

# Train the model

In [14]:
model = gensim.models.doc2vec.Doc2Vec(size=500, iter=55)

### Build the vocabulary

In [15]:
model.build_vocab(train_corpus)

In [16]:
# NOTE: SPEED UP BY INSTALLING BLAS 
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 2min 15s, sys: 3.16 s, total: 2min 18s
Wall time: 51.1 s


36239020

In [17]:
# Save model
model.save("doc2vec.model")

# Try infering

In [33]:
def topjobs(txt, n_top = 5, n_bottom = None): 
    processedText = gensim.utils.simple_preprocess(txt)
    inferred_vector = model.infer_vector(processedText)
    sims = model.docvecs.most_similar([inferred_vector])
    
    for x in sims[:n_top]: 
        job = jobs[x[0]]
        print("Job:{0};\n\tLookup Num:{1};\n\tMatch Rate:{2:.2f}".format(job['title'], x[0], x[1]))

In [34]:
# Construction worker 
construction = """Perform tasks involving physical labor at construction sites. May operate hand and power tools of all types: air hammers, earth tampers, cement mixers, small mechanical hoists, surveying and measuring equipment, and a variety of other equipment and instruments. May clean and prepare sites, dig trenches, set braces to support the sides of excavations, erect scaffolding, and clean up rubble, debris and other waste materials. May assist other craft workers."""
topjobs(construction)

Job:Philosophy and Religion Teachers, Postsecondary;
	Lookup Num:25-1126.00;
	Match Rate:0.52
Job:History Teachers, Postsecondary;
	Lookup Num:25-1125.00;
	Match Rate:0.51
Job:Political Science Teachers, Postsecondary;
	Lookup Num:25-1065.00;
	Match Rate:0.51
Job:Foreign Language and Literature Teachers, Postsecondary;
	Lookup Num:25-1124.00;
	Match Rate:0.49
Job:Mathematical Science Teachers, Postsecondary;
	Lookup Num:25-1022.00;
	Match Rate:0.48


In [146]:
# Chef 
chef = """Direct and may participate in the preparation, seasoning, and cooking of salads, soups, fish, meats, vegetables, desserts, or other foods. May plan and price menu items, order supplies, and keep records and accounts."""
topjobs(chef)

Job:English Language and Literature Teachers, Postsecondary;
	Lookup Num:324;
	Match Rate:0.62
Job:Secondary School Teachers, Except Special and Career/Technical Education;
	Lookup Num:337;
	Match Rate:0.60
Job:Kindergarten Teachers, Except Special Education;
	Lookup Num:333;
	Match Rate:0.58
Job:Art, Drama, and Music Teachers, Postsecondary;
	Lookup Num:322;
	Match Rate:0.58
Job:Economics Teachers, Postsecondary;
	Lookup Num:310;
	Match Rate:0.56


In [147]:
randomfoodwords = "food, ingredients, cook, raw"
topjobs(randomfoodwords)

Job:Political Scientists;
	Lookup Num:256;
	Match Rate:0.32
Job:Art, Drama, and Music Teachers, Postsecondary;
	Lookup Num:322;
	Match Rate:0.31
Job:English Language and Literature Teachers, Postsecondary;
	Lookup Num:324;
	Match Rate:0.30
Job:History Teachers, Postsecondary;
	Lookup Num:326;
	Match Rate:0.30
Job:Kindergarten Teachers, Except Special Education;
	Lookup Num:333;
	Match Rate:0.30


In [154]:
#statistician 
stats = """A statistician is someone who works with theoretical or applied statistics. The profession exists in both the private and public sectors. It is common to combine statistical knowledge with expertise in other subjects, and statisticians may work as employees or as statistical consultants.[1][2]"""

topjobs(stats)

Job:Pest Control Workers;
	Lookup Num:554;
	Match Rate:0.18
Job:Athletic Trainers;
	Lookup Num:483;
	Match Rate:0.18
Job:Radiologists;
	Lookup Num:424;
	Match Rate:0.17
Job:First-Line Supervisors of Non-Retail Sales Workers;
	Lookup Num:591;
	Match Rate:0.17
Job:Copy Writers;
	Lookup Num:390;
	Match Rate:0.16


In [155]:
topjobs("statistics")

Job:Stone Cutters and Carvers, Manufacturing;
	Lookup Num:913;
	Match Rate:0.20
Job:Probation Officers and Correctional Treatment Specialists;
	Lookup Num:283;
	Match Rate:0.18
Job:Occupational Therapy Assistants;
	Lookup Num:490;
	Match Rate:0.18
Job:Septic Tank Servicers and Sewer Pipe Cleaners;
	Lookup Num:740;
	Match Rate:0.18
Job:Remote Sensing Scientists and Technologists;
	Lookup Num:241;
	Match Rate:0.17
